In [1]:
# Script for investigating amplitude dependence of potential

@everywhere using Distributions
@everywhere using Base.Test
@everywhere using StatsBase
@everywhere using BenchmarkTools
using MCMCDiagnostics
@everywhere src_path = "../Source/"
@everywhere include(src_path*"types.jl")
@everywhere include(src_path*"functions_msc.jl")
@everywhere include(src_path*"functions_neighbors.jl")
@everywhere include(src_path*"functions_types.jl")
@everywhere include(src_path*"functions_energy.jl")
@everywhere include(src_path*"functions_mc.jl")
@everywhere include(src_path*"functions_thermalization.jl")
@everywhere include(src_path*"functions_observables.jl")
@everywhere include(src_path*"functions_symmetries.jl")
include(src_path*"functions_plots_and_files.jl")
using Plots
pyplot()

Plots.PyPlotBackend()

# State Array

We construct a type that acts like an array of states, but where the states are stored in a file. The stored states have to be of the same system. This is useful for reading the states generated by a MC simulation.

In [2]:
mutable struct StateArray2
    fn::AbstractString
    syst::SystConstants
    M::Int64
    pos::Array{Int64,1}
end

In [8]:
isfile("MCMC Notes 2.ipynb")

true

In [163]:
function enumerateStates(file::IOStream)
    pos = Array{Int64}(1)
    line = readline(file)
    line == "state start" || throw(error("ERROR: Could not find start of state after reading meta info"))
    pos[1] = position(file)-12
    line = readline(file)
    while line != ""
        if line == "state start"
            push!(pos, position(file)-12)
        end
        line = readline(file)
    end
    return pos
end

enumerateStates (generic function with 1 method)

In [171]:
function StateArray2(filename::AbstractString)
    if !isfile(filename)
        throw(error("ERROR: File $(filename) does not exist"))
    end
        
    syst = SystConstants(10, 10, 1/0.3^2, 0.5, 1.0, 1.0, 1.0) # Just dummy initialization.
    M₀ = 0
    pos = Array{Int64}(1)
    # Get length of existing list
    open(filename, "r") do file
        line = readline(file)
        if line != "state array"
            throw(error("ERROR: Start of file $(filename) is $(line)"))
        end
        M₀ = parse(Int64, readline(file))
        M₀ > 0 || throw(error("ERROR: Number of states in list is set to $(M₀)"))
        c_values = split(readline(file), ":")
        L = parse(Int64, c_values[1])
        L > 0 || throw(error("ERROR: States L is $(L)"))
        L₃ = parse(Int64, c_values[2])
        L₃ > 0 || throw(error("ERROR: States L₃ is $(L₃)"))
#        γ = parse(Float64, c_values[3])
        g⁻² = parse(Float64, c_values[3])
        ν = parse(Float64, c_values[4])
        κ₅ = parse(Float64, c_values[5])
        f = parse(Float64, c_values[6])
        β = parse(Float64, c_values[7])
        syst = SystConstants(L, L₃, g⁻², ν, κ₅, f, β)
        
        # Now we enumerate the positions of the states in the file and save this in the pos array.
        pos = enumerateStates(file)
    end
    StateArray2(filename, syst, M₀, pos)
end

StateArray2

In [16]:
@which a[3]

getindex(A::Array, i1::Int64) in Base at array.jl:549

In [139]:
pos = Array{Int64}(1)
push!(pos, 1)

2-element Array{Int64,1}:
 140531562649712
               1

In [36]:
for i = 1:3; @show i; end

i = 1
i = 2
i = 3


In [166]:
# Assuming an open IOStream that has been read until the start of a state,
# read the state and create corresponding state in memory.
function readState(file::IOStream, syst::SystConstants, file_pos::Int64)
    seek(file, file_pos)
    line = readline(file)
    if line != "state start"
        throw(DomainError())
    end
    L = syst.L
    L₃ = syst.L₃
    lattice = Array{LatticeSite, 3}(L,L,L₃)
    for z_pos = 1:L₃, h_pos = 1:L, v_pos = 1:L
        ϕ_values = split(readline(file), ":")
        A₁ = parse(Float64, ϕ_values[1])
        A₂ = parse(Float64, ϕ_values[2])
        A₃ = parse(Float64, ϕ_values[3])
        θ⁺ = parse(Float64, ϕ_values[4])
        θ⁻ = parse(Float64, ϕ_values[5])
        u⁺ = parse(Float64, ϕ_values[6])
        u⁻ = parse(Float64, ϕ_values[7])
        lattice[v_pos,h_pos,z_pos] = LatticeSite([A₁, A₂, A₃], θ⁺, θ⁻, u⁺, u⁻)
    end

    nbl = latticeNeighbors(lattice,L,L₃)
    nnbl = latticeNextNeighbors(lattice,L,L₃)
    nnnbl = latticeNNNeighbors(lattice,L,L₃)

    return State(lattice, syst, nbl, nnbl, nnnbl)
end

readState (generic function with 3 methods)

In [5]:
import Base.getindex

In [169]:
# Take an index and a state array, and output the corresponding state in the state-list.
function getindex(sa::StateArray2, i::Int64)
    # Checking that integer wanted is within the size of the array
    if i>sa.M || i < 1
        throw(error("ERROR: $(i) is not a valid array index in array of length $(sa.M)"))
    end
    # At this point in the code then i ∈ [1, M]
    ψ = State(1, sa.syst) # Initialize state
    N = sa.syst.L^2*sa.syst.L₃
    open(sa.fn, "r") do file
        # Find the start position of state # i
        pos = sa.pos[i]
        
        # Now we read the state that is at this position.
        ψ = readState(file, sa.syst, pos)
    end
    return ψ
end

getindex (generic function with 274 methods)

In [50]:
ψ_list = 0
gc()

In [3]:
syst = SystConstants(6, 6, 1/0.3^2, 0.5, 1.0, 1.0, 1.0)
ψ_list = Array{State}(3)
for m = 1:3
    ψ_list[m] = State(2, syst)
end

In [4]:
save(ψ_list, "test.state")

1

In [172]:
teSA = StateArray2("test.state");

In [174]:
ψ = teSA[1];

In [34]:
teSA[2] == ψ₂

true

In [175]:
teSA[1] == teSA[2]

false

In [7]:
function printPosition(file::IOStream)
    @show position(file)
end

printPosition (generic function with 1 method)

In [5]:
f = open("test.state", "r")

IOStream(<file test.state>)

In [10]:
printPosition(f)

position(file) = 12


12

In [43]:
close(f)

In [155]:
readline(f)

"state start"

In [160]:
position(f)

73

In [153]:
position(f)

73

In [164]:
seek(f, 61)

IOStream(<file test.state>)

In [165]:
enumerateStates(f)

3-element Array{Int64,1}:
    61
 28352
 56667

In [167]:
typeof(readState(f, syst, 61))

State

In [126]:
line = readline(f)
while line != ""
    if line == "state start"
        @show position(f)
    end
    line = readline(f)
end

position(f) = 73
position(f) = 28364
position(f) = 56679


In [131]:
seek(f, 73)
readline(f)

"-2.3801091798928997:-4.165923301343946:-1.9676768072055761:1.8965311506309228:3.7900556357019375:0.7016431350638241:1.5244380513954912"